In [ ]:
import os
import pylab as plt
from spacepy import pycdf
import numpy as np
from datetime import *
import pandas as pd
os.environ['CDF_LIB'] = '/home/dedasilv/.local/cdf/lib'

cdf = pycdf.CDF('data/Dec20_2015_storm/omni_hro_1min_20151201_v01.cdf')

In [ ]:
t = cdf['Epoch'][:]
n = cdf['proton_density'][:]
Bz = cdf['BZ_GSM'][:]
Vx = cdf['Vx'][:]
Vy = cdf['Vy'][:]
Vz = cdf['Vz'][:]
symh = cdf['SYM_H'][:]

In [ ]:
n[n > 999] = np.nan
Bz[Bz > 999] = np.nan
Vx[Vx > 999] = np.nan
Vy[Vy > 999] = np.nan
Vz[Vz > 999] = np.nan
mask = (t > datetime(2015, 12, 18)) & (t < datetime(2015, 12, 23))
speed = np.sqrt(Vx**2 + Vy**2 + Vz**2)

In [ ]:
import glob

files = glob.glob('data/Dec20_2015_storm_F*.csv')
dfs = []
for file in files:
    df = pd.read_csv(file, parse_dates=['start_time', 'end_time'])
    dfs.append(df)
    
df_sel = pd.concat(dfs).reset_index()

In [ ]:
df_sel

In [ ]:
small_size, med_size, big_size = 16, 18, 20
plt.rc('font', size=small_size)          # controls default text sizes
plt.rc('axes', titlesize=small_size)     # fontsize of the axes title
plt.rc('axes', labelsize=med_size)       # fontsize of the x and y labels
plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=small_size)    # legend fontsize
plt.rc('figure', titlesize=big_size)     # fontsize of the figure title

fig, axes = plt.subplots(3, 1, figsize=(18, 12), sharex='all')

fig.suptitle('December 20, 2015 Storm\nDispersion Events', fontsize=30, fontweight='bold')

axes[0].plot(t[mask], symh[mask], color='C4')
axes[0].axhline(0, color='black')
axes[0].set_ylabel('SYM-H Index (nT)', fontsize=20, color='C4')
axes[1].plot(t[mask], Bz[mask], color='C1')
axes[1].set_ylabel('IMF Bz (nT)', fontsize=20,  color='C1',)
axes[1].axhline(0, color='black')
axes[2].plot(t[mask], 8.36e-7 * n[mask] * speed[mask]**2, color='C2')
axes[2].set_ylabel('IMF Dynamic\nPressure (nPa)', fontsize=20, color='C2')
#axes[2].set_xlim(datetime(2015, 12, 18, 12), datetime(2015, 12, 22, 10))
for ax in axes:
    ax.grid(True, color='gray', linestyle='dashed')

    for i, row in df_sel.iterrows():
        if i == 0:
            kwargs = {'label': 'Dispersion Event'}
        else:
            kwargs = {}
        ax.axvline(row.start_time, color='red', **kwargs)
    
    ax.legend()